In [ ]:
%load_ext autoreload
%autoreload 2
import numpy
import pandas
import io
import uuid
import psycopg.sql
import pyarrow
import pyarrow.parquet

import jobqueue
from jobqueue.connection_manager import ConnectionManager



import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import sys

import dmp.keras_interface.model_serialization as model_serialization
from dmp.task.experiment.training_experiment.training_epoch import TrainingEpoch

pd.options.display.max_seq_items = None
np.set_printoptions(threshold=sys.maxsize)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
# from enum import Enum, auto

# class ColumnSource(Enum):
#     run_column=auto()
#     run
#     run_column=auto()
#     run_column=auto()
#     run_column=auto()
#     run_column=auto()

from dataclasses import dataclass



@dataclass
class Column:
    name : str
    pandas_type : str


In [ ]:
'''
parent/root runs:
select
	*
from
	run_data d
where
	command @> '{"experiment":{"type":"LTHExperiment"}}'
limit 10
;
'''

In [ ]:
"""
select
 	command->'experiment'->'dataset'->'name' dataset,
	command->'experiment'->'pruning'->'method'->'pruning_rate' pruning_rate,
	id,
	experiment_id,
	command,
	history
from run
where TRUE
	AND queue >=0
	AND status >= 2
	AND command @> '{"experiment":{
		"type":"IterativePruningExperiment",
		"pruning":{
			"method":{
				"type":"MagnitudePruner"
				}
		}}}'::jsonb
LIMIT 100;
"""

In [3]:
def flatten_json(json_obj, parent_key="", separator="_"):
    flattened = {}
    for key, value in json_obj.items():
        new_key = f"{parent_key}{separator}{key}" if parent_key else key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key, separator=separator))
        else:
            flattened[new_key] = value
    return flattened


# AND id IN ('65ef3809-d900-4f86-9591-ff434351b1bc', 'b67d2005-70dc-46e8-969f-3021234e4908')
# IN ('95475c6f-2cdd-4d0e-96b7-147ead863143', 'dec62d55-92e2-45a1-939b-321dfc7617a5')

experiment_id = 0
run_command = {}
credentials = jobqueue.load_credentials("dmp")
dataframes = []
with ConnectionManager(credentials) as connection:
    with connection.cursor(binary=True) as cursor:
        query = psycopg.sql.SQL(
            """
SELECT
    id,
	experiment_id,
	command,
	history
FROM
    run r
WHERE TRUE
    AND id = 'ffd1a61a-c567-4978-b912-2f53cec3d77f'
;
"""
        )

        cursor.execute(query, binary=True)

        for row in cursor.fetchall():
            run_id = row[0]
            experiment_id = row[1]

            with io.BytesIO(row[3]) as buffer:
                history = pyarrow.parquet.read_table(
                    pyarrow.PythonFile(buffer, mode="r")
                ).to_pandas()

            run_command = row[2]
            from dmp.marshaling import marshal

            run = marshal.demarshal(run_command)

            experiment_data = {
                "run_id": [run_id],
                "experiment_id":[experiment_id],
            }

            flat = flatten_json(run_command)
            for k, v in flat.items():
                if isinstance(v, list):
                    continue
                experiment_data[k] = [v]
                # history[k] = v

            experiment_df = pd.DataFrame(experiment_data)
            history = history.join(experiment_df, how='cross')
            dataframes.append(history)

history = pd.concat(dataframes, ignore_index=True, axis=0)

history.sort_values(
    ["experiment_id", "run_id", "fit_number", "fit_epoch", ], inplace=True
)

# history.describe()
sorted(history["run_id"].unique())

[UUID('ffd1a61a-c567-4978-b912-2f53cec3d77f')]

In [4]:
history[history['fit_number'] == 0]

,train_loss,trained_loss,fit_epoch,test_ms,validation_accuracy,seed_number,train_accuracy,epoch_startms_ms,retained,test_accuracy,masked_parameter_count,epoch,trainms_ms,fit_number,free_parameter_count,trained_accuracy,trained_ms,test_loss,validation_loss,test_categorical_hinge_extended,trained_categorical_hinge_extended,train_kullback_leibler_divergence_extended,train_cosine_similarity_extended,test_kullback_leibler_divergence_extended,test_cosine_similarity_extended,validation_categorical_hinge_extended,trained_kullback_leibler_divergence_extended,trained_cosine_similarity_extended,validation_kullback_leibler_divergence_extended,validation_cosine_similarity_extended,train_categorical_hinge_extended,validation_compile_metrics,trained_compile_metrics,train_kl_divergence,validation_kl_divergence,train_compile_metrics,test_compile_metrics,run_id,experiment_id,type,config_data_batch,config_data_run_id_type,config_data_run_id_label,config_data_run_id_value,config_data_context_num_cpus,config_data_context_num_gpus,config_data_context_queue_id,config_data_context_num_nodes,config_data_context_worker_id_type,config_data_context_worker_id_value,config_data_context_gpu_memory,config_data_context_tensorflow_strategy,config_data_git_hash,config_data_platform,config_data_host_name,config_data_slurm_job_id,config_data_python_version,config_data_tensorflow_version,config_seed,config_type,config_model_saving_type,config_model_saving_fixed_interval,config_model_saving_fixed_threshold,config_model_saving_exponential_rate,config_model_saving_save_initial_model,config_model_saving_save_trained_model,config_record_times,config_rewind_run_id_type,config_rewind_run_id_value,config_resume_checkpoint_type,config_resume_checkpoint_epoch_type,config_resume_checkpoint_epoch_epoch,config_resume_checkpoint_epoch_marker,config_resume_checkpoint_epoch_fit_epoch,config_resume_checkpoint_epoch_fit_number,config_resume_checkpoint_epoch_sequence_number,config_resume_checkpoint_run_id_type,config_resume_checkpoint_run_id_label,config_resume_checkpoint_run_id_value,config_resume_checkpoint_load_mask,config_resume_checkpoint_load_optimizer,config_prune_first_iteration,config_record_post_training_metrics,run_entry,experiment_fit_epochs,experiment_fit_batch_size,experiment_data_lmc,experiment_data_batch,experiment_data_group,experiment_data_ml_task,experiment_data_model_name,experiment_data_supergroup,experiment_data_model_family,experiment_data_resnet_depth,experiment_data_data_set_size,experiment_data_test_set_size,experiment_data_train_set_size,experiment_data_num_free_parameters,experiment_data_validation_set_size,experiment_loss_class,experiment_type,experiment_model_type,experiment_model_input_name,experiment_model_input_type,experiment_model_output_type,experiment_model_output_units,experiment_model_output_use_bias,experiment_model_output_activation,experiment_model_output_bias_constraint,experiment_model_output_bias_initializer,experiment_model_output_bias_regularizer,experiment_model_output_kernel_constraint,experiment_model_output_kernel_initializer_class,experiment_model_output_kernel_regularizer,experiment_model_output_activity_regularizer,experiment_dataset_name,experiment_dataset_type,experiment_dataset_method,experiment_dataset_source,experiment_dataset_test_split,experiment_dataset_label_noise,experiment_dataset_validation_split,experiment_pruning_type,experiment_pruning_method_type,experiment_pruning_method_pruning_rate,experiment_pruning_new_seed,experiment_pruning_iterations,experiment_pruning_rewind_epoch_type,experiment_pruning_rewind_epoch_epoch,experiment_pruning_rewind_epoch_marker,experiment_pruning_rewind_epoch_fit_epoch,experiment_pruning_rewind_epoch_fit_number,experiment_pruning_rewind_epoch_sequence_number,experiment_pruning_rewind_optimizer,experiment_pruning_max_epochs_per_iteration,experiment_optimizer_class,experiment_optimizer_learning_rate_class,experiment_precision,experiment_early_stopping_class,experiment_early_stopping_monitor

In [ ]:
print(history)

In [ ]:
experiment = run.experiment
dataset, metrics, loss_metric = experiment._load_and_prepare_dataset()
network = experiment._make_network(experiment.model)
keras_model = experiment
network.summary()

In [ ]:
print(network.structure.summary())

In [ ]:
print(history.columns)

In [ ]:
print((history['epoch'] - history['fit_epoch']).to_string())

In [ ]:
history['experiment_pruning_method_pruning_rate'].loc[0]

In [ ]:
from pprint import pprint
pprint(run_command)

In [ ]:
fig = plt.figure(dpi=200, figsize=(6,4))


# plt.semilogy(history['epoch'], history['test_loss'], '-og', markersize=3.0, alpha=.7)
plt.plot(history['epoch'], history['fit_number'], '-xr', markersize=3.0, alpha=.7)
plt.show()
plt.plot(history['fit_epoch'], history['fit_number'], '--g', markersize=3.0, alpha=.7)


In [ ]:
retained = history.loc[history["retained"] == True]
for run_id, run in retained.groupby("run_id"):
    plt.semilogy(run["epoch"], run["test_loss"], "-", markersize=3.0, alpha=0.7)

In [ ]:
np.set_printoptions(
    edgeitems=3,
    infstr="inf",
    linewidth=75,
    nanstr="nan",
    precision=8,
    suppress=False,
    threshold=10000,
    formatter=None,
)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

history[
    [
        "epoch",
        "fit_number",
        "fit_epoch",
        "retained",
        "validation_loss",
        "test_loss",
        "masked_parameter_count",
    ]
].head(100)

In [ ]:
history["masked_parameter_count"]

In [ ]:
for run_id, run in retained.groupby("run_id"):
    plt.loglog(
        run["epoch"], run["masked_parameter_count"], "-", markersize=3.0, alpha=0.7
    )

In [ ]:
for run_id, run in history.groupby("run_id"):
    retained = run.loc[run["retained"] == True]
    trained = retained.loc[
        retained.groupby(["fit_number"])["epoch"].transform("idxmax")
    ]

    plt.loglog(
        100
        * (1 - (trained["masked_parameter_count"]) / trained["free_parameter_count"]),
        trained["test_loss"],
        ".",
        markersize=3.0,
        alpha=0.7,
    )
    plt.xlabel("% Retained")
    plt.ylabel("Loss")

    # plt.show()
    # plt.semilogx(
    #     100 * (1 - (trained["masked_parameter_count"]) / trained["free_parameter_count"]),
    #     100 * (1 - trained["test_accuracy"]),
    #     ".g",
    #     markersize=3.0,
    #     alpha=0.7,
    # )
    # plt.xlabel("% Retained")
    # plt.ylabel("% Error")
    # plt.semilogy(retained['epoch'], retained['trained_loss'], '--g', markersize=3.0, alpha=.7)

In [ ]:

x = retained.loc[retained["fit_number"] < 10]
for fit_number, fit in x.groupby(["fit_number"]):
    plt.semilogy(
        fit["fit_epoch"],
        fit["test_loss"],
        "-",
        markersize=3.0,
        alpha=0.7,
    )

# plt.semilogy(retained['fit_epoch'], retained['test_loss'], '-xr', markersize=3.0, alpha=.7)

In [ ]:
plt.loglog(
    retained["epoch"],
    (retained["free_parameter_count"] - retained["masked_parameter_count"]) / retained["free_parameter_count"],
    "-b",
    markersize=3.0,
    alpha=0.7,
)

In [ ]:
retained = history.loc[history["retained"] == True]
plt.loglog((1 + retained['masked_parameter_count']) / retained['free_parameter_count'], retained['test_loss'], 'xr', markersize=3.0, alpha=.7)

In [ ]:
import h5py as h5
import hdf5plugin


def extract_trajectories(parameter_dataset, parameter_indicies, epochs):
    parameter_values = parameter_dataset[parameter_indicies, :]

    trajectories = np.ndarray(shape=(len(parameter_indicies), len(epochs)))
    for i, (epoch, sequence_number) in enumerate(epochs):
        parameters_at_epoch = parameter_values[:, sequence_number]
        trajectories[:, i] = parameters_at_epoch

    return trajectories


for run_id, run in history.groupby("run_id"):
    retained = run.loc[run["retained"] == True]
    # trained = retained.loc[
    #     retained.groupby(["fit_number"])["epoch"].transform("idxmax")
    # ]

    # scp eagle:/projects/modularai/dmp/dmp/model_data/c1a3792d-c30d-4b55-9ca4-d1bbcc59518c.h5 .
    dmp_path = "/home/ctripp/dmp/dmp/"
    model_data_dir = os.path.join(dmp_path, "model_data")
    checkpoint_path = os.path.join(model_data_dir, f"{run_id}.h5")
    with h5.File(checkpoint_path, "r") as h5_file:
        (
            epoch_dataset,
            parameter_dataset,
            optimizer_datasets,
        ) = model_serialization.get_datasets_from_model_file(h5_file, None)

        global_epoch = epoch_dataset[0, :]
        fit_number = epoch_dataset[1, :]
        fit_epoch = epoch_dataset[2, :]
        epoch_marker = epoch_dataset[3, :]

        epochs = []
        for i in range(epoch_dataset.shape[1]):
            epoch = TrainingEpoch(
                global_epoch[i], fit_number[i], fit_epoch[i], epoch_marker[i]
            )

            if retained.loc[
                (retained["epoch"] == epoch.epoch)
                & (retained["fit_number"] == epoch.fit_number)
                & (retained["fit_epoch"] == epoch.fit_epoch)
            ].empty:
                continue
            epochs.append(
                (
                    epoch,
                    i,
                )
            )
        epochs.sort()

        # initial_epochs = [tup for tup in epochs if tup[0].fit_epoch == 0]
        # final_epochs = [tup for tup in epochs if tup[0].marker != 0]

        epoch_ranges = []
        fit_number = None
        for epoch, sequence_number in epochs:
            if (
                len(epoch_ranges) == 0
                or epoch_ranges[-1][-1][0].fit_number != epoch.fit_number
            ):
                plt.axvline(x=epoch.epoch, color="k", alpha=0.7)
                epoch_ranges.append([])
            epoch_ranges[-1].append((epoch, sequence_number))

        # samples = []
        # for epoch_range in epoch_ranges:
        #     trajectories
        # parameter_indicies = list(range(40))

        num_samples = 1000
        num_samples = min(num_samples, parameter_dataset.shape[0])
        parameter_indicies = np.random.choice(
            parameter_dataset.shape[0], replace=False, size=num_samples
        )
        parameter_indicies = list(np.sort(parameter_indicies))

        trajectories = extract_trajectories(
            parameter_dataset, parameter_indicies, epochs
        )

        for i in range(trajectories.shape[0]):
            plt.plot(
                [epoch.epoch for epoch, sequence_number in epochs],
                trajectories[i, :],
                ".-",
                alpha=0.5,
            )

        plt.yscale('symlog')
        # plt.xscale('log')
        plt.xlim(10,725)
        plt.xlabel("Training Epoch")
        plt.ylabel("Parameter Value")
        plt.ylim(-.4,.4)
        plt.yticks([-0.3, 0, 0.3], ['-0.3', '0', '0.3'])

        plt.show()
        # for epoch, sequence_number in epochs:
        # if epoch.fit_epoch == 0:
        #     plt.axvline(x=epoch.epoch, color="r", alpha=0.5)

In [ ]:
import h5py as h5
import hdf5plugin


def extract_trajectories(parameter_dataset, parameter_indicies, epochs):
    parameter_values = parameter_dataset[parameter_indicies, :]

    trajectories = np.ndarray(shape=(len(parameter_indicies), len(epochs)))
    for i, (epoch, sequence_number) in enumerate(epochs):
        parameters_at_epoch = parameter_values[:, sequence_number]
        trajectories[:, i] = parameters_at_epoch

    return trajectories


for run_id, run in history.groupby("run_id"):
    retained = run.loc[run["retained"] == True]
    # trained = retained.loc[
    #     retained.groupby(["fit_number"])["epoch"].transform("idxmax")
    # ]

    # scp eagle:/projects/modularai/dmp/dmp/model_data/c1a3792d-c30d-4b55-9ca4-d1bbcc59518c.h5 .
    dmp_path = "/home/ctripp/dmp/dmp/"
    model_data_dir = os.path.join(dmp_path, "model_data")
    checkpoint_path = os.path.join(model_data_dir, f"{run_id}.h5")
    with h5.File(checkpoint_path, "r") as h5_file:
        (
            epoch_dataset,
            parameter_dataset,
            optimizer_datasets,
        ) = model_serialization.get_datasets_from_model_file(h5_file, None)

        global_epoch = epoch_dataset[0, :]
        fit_number = epoch_dataset[1, :]
        fit_epoch = epoch_dataset[2, :]
        epoch_marker = epoch_dataset[3, :]

        epochs = []
        for i in range(epoch_dataset.shape[1]):
            epoch = TrainingEpoch(
                global_epoch[i], fit_number[i], fit_epoch[i], epoch_marker[i]
            )

            if retained.loc[
                (retained["epoch"] == epoch.epoch)
                & (retained["fit_number"] == epoch.fit_number)
                & (retained["fit_epoch"] == epoch.fit_epoch)
            ].empty:
                continue
            epochs.append(
                (
                    epoch,
                    i,
                )
            )
        epochs.sort()

        initial_epochs = [tup for tup in epochs if tup[0].fit_epoch == 0]
        final_epochs = [tup for tup in epochs if tup[0].marker != 0]

        # parameter_indicies = list(range(40))
        num_samples = 20
        num_samples = min(num_samples, parameter_dataset.shape[0])
        parameter_indicies = np.random.choice(
            parameter_dataset.shape[0], replace=False, size=num_samples
        )
        parameter_indicies = list(np.sort(parameter_indicies))

        initial_trajectories = extract_trajectories(
            parameter_dataset, parameter_indicies, initial_epochs
        )
        final_trajectories = extract_trajectories(
            parameter_dataset, parameter_indicies, final_epochs
        )

        # global_epoch = np.fromiter(
        #     (epoch.epoch for epoch, sequence_number in epochs), dtype=int
        # )
        # for i in range(trajectories.shape[0]):
        #     plt.loglog(global_epoch, trajectories[i, :], "-", alpha=0.5)
        # for i in range(initial_trajectories.shape[0]):
        #     plt.semilogy(
        #         list(range(len(epochs))), initial_trajectories[i, :], "-", alpha=0.5
        #     )

        for i in range(final_trajectories.shape[0]):
            plt.loglog(
                list(range(len(final_epochs))),
                np.abs(final_trajectories[i, :]),
                "-",
                alpha=0.5,
            )

        plt.show()
        # for epoch, sequence_number in epochs:
        # if epoch.fit_epoch == 0:
        #     plt.axvline(x=epoch.epoch, color="r", alpha=0.5)

In [ ]:
print(epoch_dataset[:,:])

In [ ]:
with h5.File(checkpoint_path, "r") as h5_file:
    (
        epoch_dataset,
        parameter_dataset,
        optimizer_datasets,
    ) = model_serialization.get_datasets_from_model_file(h5_file, None)

    epoch = TrainingEpoch(epoch_dataset[0,0],
                          epoch_dataset[1,0],
                          epoch_dataset[2,0],
                          epoch_dataset[3,0],)

model = experiment._make_model_from_network(network, metrics)
model_serialization.load_parameters(
        model.network.structure,
        model.keras_network.layer_to_keras_map,
        None,
        checkpoint_path,
        epoch,
        load_mask=True,
    )

In [ ]:
for layer in model.network.structure.layers:
    print(layer)

In [ ]:
model.keras_model.summary()

In [ ]:
import tensorflow.keras as keras
for layer in model.network.structure.layers:
    layer_info = model.keras_network.layer_to_keras_map.get(layer, None)
    if layer_info is None:
        continue

    keras_layer = layer_info.keras_layer
    if keras_layer is None or not isinstance(keras_layer, keras.layers.Layer):
        continue

    for i, variable in enumerate(keras_layer.variables):  # type: ignore
        print(f'layer:{layer} variable {i} with name {variable.name} and shape {variable.shape}')

In [ ]:
fig = plt.figure(dpi=200, figsize=(6,4))

plt.plot(history['epoch'], history['test_loss'], '--x', markersize=3.0, alpha=.7)
new_iterations = history[history['model_epoch'] == 0]
plt.plot(by_epoch['epoch'], by_epoch['test_loss_cmin_quantile_50'], 'og', markersize=3.0, alpha=.7)

plt.plot(new_iterations['epoch'], new_iterations['test_loss'], 'or', markersize=3.0, alpha=.7)


In [ ]:
history.columns

In [ ]:
fig = plt.figure(dpi=200, figsize=(6,4))

plt.plot(history['epoch'], history['free_parameter_count'], '--x', markersize=3.0, alpha=.7)


In [ ]:

credentials = jobqueue.load_credentials('dmp')
with ConnectionManager(credentials) as connection:
    with connection.cursor(binary=True) as cursor:
        query = psycopg.sql.SQL("""
SELECT
    by_epoch
FROM
    experiment2 e
WHERE TRUE
    AND e.experiment_id = 'b0d415cf-f228-c439-0480-3337c543c2e4'
LIMIT 1;
""")

        cursor.execute(query, binary=True)
        row = cursor.fetchone()
        with io.BytesIO(row[0]) as buffer:
            by_epoch = pyarrow.parquet.read_table(
                pyarrow.PythonFile(buffer, mode='r')).to_pandas()


by_epoch.describe()
by_epoch

In [ ]:
df.columns

In [ ]:
df['run_history'].iloc[0]['train_loss']


In [ ]:
run_df = df['run_history'].iloc[0]
run_df['trained_loss'] - run_df['train_loss']